In [1]:
import pandas as pd
import re
from collections import Counter
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def assignDivision(divisionsDict, divisions):
    for key in divisionsDict:
        div = re.split(r' |/|\+|,|-', divisionsDict[key])
        for d in divisions: #loop through list with known divisions
            if d in div:
                assignedDiv = d
                break
        divisionDict[key] = {'Assigned division': assignedDiv}
    return divisionDict

In [3]:
#load 
researchers = pd.read_pickle("Files/Dataframes/researchers_wGender.pkl")
researchers['Division'] = researchers.Division.fillna('Unknown')

In [4]:
#APS Units
sponsorFile = open("Files/sponsoringUnits.txt", "r").read()
sponsorList = sponsorFile.replace('\n', ';').split(';')
sponsor_abr = re.findall(r'\((.*)\)', sponsorFile) #abbreviations for the sponsoring units
sponsorNames = re.findall(r'(.*) \(', sponsorFile) #ful name of sponsoring unit

In [5]:
researchers['AllDivisions'] = researchers.Division.str.split(r' |/|\+|,|-')
researcher_divisions = researchers.explode('AllDivisions') 

In [6]:
#df with only divisions known from the APS homepage
researcher_knownDivisions = researcher_divisions[researcher_divisions.AllDivisions.isin(sponsor_abr)]

In [7]:
sponsorDict = dict(zip(sponsor_abr, sponsorNames))

In [8]:
researcher_knownDivisions['Division'] = researcher_knownDivisions['AllDivisions'].apply(lambda x: sponsorDict.get(x))
#researcher_knownDivisions["Division"].apply(lambda x: d.get(x))

In [11]:
#remove division column with abbreviations
researcher_knownDivisions = researcher_knownDivisions.drop('AllDivisions', axis = 1)


In [12]:
researcher_knownDivisions.to_pickle("Files/Dataframes/researchersFinal.pkl") 